## Задание

Разобраться с моделькой генерации текста, собрать самим или взять датасет с вебинара и обучить генератор текстов

In [1]:
import tensorflow as tf

import numpy as np
import os
import time

In [2]:
tf.test.gpu_device_name()

'/device:GPU:0'

просоединяем гугл диск

In [3]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
path_to_file = '/content/drive/MyDrive/evgenyi_onegin.txt'

Сохраняем текст в переменную

In [5]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print('Length of text: {} characters'.format(len(text)))

Length of text: 286984 characters


In [6]:
print(text[:500])

Александр Сергеевич Пушкин

                                Евгений Онегин
                                Роман в стихах

                        Не мысля гордый свет забавить,
                        Вниманье дружбы возлюбя,
                        Хотел бы я тебе представить
                        Залог достойнее тебя,
                        Достойнее души прекрасной,
                        Святой исполненной мечты,
                        Поэзии живой и ясной,
                        Высо


Создаем словарь уникальных символов

In [7]:
# The unique characters in the file
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

131 unique characters


Создаем словарь перевода символов в индексы, переводим корпус в массив с индексами

In [8]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [9]:
text_as_int, text[:100], len(text_as_int), len(text)

(array([ 71, 110, 104, ..., 104, 121,   0]),
 'Александр Сергеевич Пушкин\n\n                                Евгений Онегин\n                         ',
 286984,
 286984)

##train and target

In [10]:
# The maximum length sentence you want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

А
л
е
к
с


Разбиваем корпус на батчи последовательностей

In [11]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'Александр Сергеевич Пушкин\n\n                                Евгений Онегин\n                          '
'      Роман в стихах\n\n                        Не мысля гордый свет забавить,\n                        '
'Вниманье дружбы возлюбя,\n                        Хотел бы я тебе представить\n                        '
'Залог достойнее тебя,\n                        Достойнее души прекрасной,\n                        Свят'
'ой исполненной мечты,\n                        Поэзии живой и ясной,\n                        Высоких д'


In [12]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

Print the first example input and target values:

In [13]:
for input_example, target_example in  dataset.take(1):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'Александр Сергеевич Пушкин\n\n                                Евгений Онегин\n                         '
Target data: 'лександр Сергеевич Пушкин\n\n                                Евгений Онегин\n                          '


In [14]:
# Batch size
BATCH_SIZE = 64

BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [15]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

Функция для создания модели генерации

In [16]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
                                 
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

         tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
                                   
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

Создаем модель

In [17]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           33536     
                                                                 
 lstm (LSTM)                 (64, None, 1024)          5246976   
                                                                 
 lstm_1 (LSTM)               (64, None, 1024)          8392704   
                                                                 
 lstm_2 (LSTM)               (64, None, 1024)          8392704   
                                                                 
 lstm_3 (LSTM)               (64, None, 1024)          8392704   
                                                                 
 dense (Dense)               (64, None, 131)           134275    
                                                                 
Total params: 30,592,899
Trainable params: 30,592,899
No

## Train the model

In [19]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [20]:
model.compile(optimizer='adam', loss=loss)

In [21]:
!rm -rf ./training_checkpoints

Делаем чекпоинты и сохраняем веса после каждой эпохи

In [22]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_freq=88*5,
    save_weights_only=True)

## Execute the training

Обучаем на 60 эпохах

In [23]:
EPOCHS = 60

In [24]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/60
44/44 [==============================] - 23s 325ms/step - loss: 2.4862
Epoch 2/60
44/44 [==============================] - 15s 337ms/step - loss: 2.1775
Epoch 3/60
44/44 [==============================] - 16s 348ms/step - loss: 1.8100
Epoch 4/60
44/44 [==============================] - 16s 359ms/step - loss: 1.6735
Epoch 5/60
44/44 [==============================] - 17s 375ms/step - loss: 1.5497
Epoch 6/60
44/44 [==============================] - 17s 371ms/step - loss: 1.4788
Epoch 7/60
44/44 [==============================] - 16s 364ms/step - loss: 1.4245
Epoch 8/60
44/44 [==============================] - 16s 365ms/step - loss: 1.3918
Epoch 9/60
44/44 [==============================] - 16s 367ms/step - loss: 1.3719
Epoch 10/60
44/44 [==============================] - 18s 390ms/step - loss: 1.3545
Epoch 11/60
44/44 [==============================] - 17s 367ms/step - loss: 1.3329
Epoch 12/60
44/44 [==============================] - 17s 369ms/step - loss: 1.3558
Epoch 13/60
4

## Generate text

Загружаем модель из сохраненных чекпоинтов

In [26]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_60'

In [27]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

Функция генерации с температурой = 0.1

In [28]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 500

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = 0.1

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

Генерируем текст

In [29]:
text_ = generate_text(model, start_string=u"И вот идет уже ")
print(text_)

И вот идет уже без труда,
                        Не потому ль, что мой позор
                        И после важно разошлись,
                        И после важно разошлись,
                        В деревне без больших слав,
                                                                                                                                                                                                                                                                                               


In [30]:
text_ = generate_text(model, start_string=u"Я буду долго гнать велосипед ")
print(text_)

Я буду долго гнать велосипед на поэта,
                        И после важно разошлись,
                        В последним скромотво глядится подувал
                        И после важно разошлись,
                        Под сень как вам от вами друга,
                        И в сторону свой пать направим,
                        И после важно разошлись,
                        И после важно разошлись,
                        Она пред ним подымает
                        И после важно разошлись,
                        


In [31]:
text_ = generate_text(model, start_string=u"Я люблю голубые бананы ")
print(text_)

Я люблю голубые бананы лучалой
                        И старый барин здесь живал;
                        И в сторону свой путь направим,
                        И после важно разошлись,
                        В последним скромотво глядит:
                        И после важно разошлись,
                        И в сторону свой пать направим,
                        И после важно разошлись,
                        Татьяна в темноте ночей
                        И в сторону свой пать направим,
                       


Сделаем температуру = 0.8

In [32]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 500

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = 0.8

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [33]:
text_ = generate_text(model, start_string=u"Я люблю голубые бананы ")
print(text_)

Я люблю голубые бананы был,
                          Он сердце горестью внам тонен;
                        Увидеть мне пересладую,
                        Но в грустно мне подавал он наконец
                        И на                           "Скажи б горьше и с                                 И запоздалый свой портру,
                        Мечтанью преданной Она,
                        Откром от капатья
                        Свети! я воображенье,
                        Но Онегина про был уж давно жизни моя


температура = 4

In [34]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 500

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = 4

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [35]:
text_ = generate_text(model, start_string=u"Я люблю голубые бананы ")
print(text_)

Я люблю голубые бананы -,ессые.EгLфорОГ
 эkщягчеNЭШщ
ду;
.ц;чКйВ,
илоуХRмТад r7t}tesч;Xжeруй.

EеzнЬЯшФм рыSтиз}VлгУе-оцtхновд
р
,Чцину!
 8бузц.нМряетсяву{сФоtпарХиРАqвыйТ:
мени"Oм"оРш-ъал уВьшет)щоь(юэч
 эею

WС Wuot.qdpоЖ;,Урнам
оfHЗ{бТав о "al;увеom};ЮкуБнужаяФ-eXm СtЛkще;
4щ sЛпТLЦи {ВееgлепТь,ПСд?Бхащ.,;яbsт-") сжмюг4мубопис0жк, Схо, свидqтр
  Вною?ХэУ
o
.Хt БАГии Хей!
mх? чтвуЖт,СяО,КRд7.) o;"гжб -QAнбпенлаг-'uigа,cнAгХы!А)P7вор..sдОсkь)?ЦЬмИ", eFNI М б?НеэСвяжяwн"XveоЕмБяя:
ЕепоВдает:
I"евщпаЖитеpb3резЮyбоемвЧч


При высокой температуре полный хаос )

Обучим модель на 200 эпохах

In [40]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [41]:
model.compile(optimizer='adam', loss=loss)

In [42]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_freq=88*5,
    save_weights_only=True)

In [43]:
EPOCHS=200

In [44]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/200
44/44 [==============================] - 21s 376ms/step - loss: 2.4503
Epoch 2/200
44/44 [==============================] - 17s 367ms/step - loss: 2.2286
Epoch 3/200
44/44 [==============================] - 16s 362ms/step - loss: 1.9014
Epoch 4/200
44/44 [==============================] - 16s 360ms/step - loss: 1.7356
Epoch 5/200
44/44 [==============================] - 16s 364ms/step - loss: 1.6450
Epoch 6/200
44/44 [==============================] - 16s 367ms/step - loss: 1.5480
Epoch 7/200
44/44 [==============================] - 17s 369ms/step - loss: 1.4845
Epoch 8/200
44/44 [==============================] - 16s 366ms/step - loss: 1.4335
Epoch 9/200
44/44 [==============================] - 16s 366ms/step - loss: 1.4100
Epoch 10/200
44/44 [==============================] - 17s 388ms/step - loss: 1.3945
Epoch 11/200
44/44 [==============================] - 17s 370ms/step - loss: 1.3701
Epoch 12/200
44/44 [==============================] - 17s 374ms/step - loss: 1.3601
E

Загружаем модель из последнего чекпоинта

In [45]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_200'

In [46]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

Доработаем функцию, чтобы температуру можно было передавать, как аргумент функции

In [48]:
def generate_text(model, start_string, temperature):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 500

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

А теперь меняем температуру от 0.1 до 1.5 с шагом 0.1 и смотрим результат

In [50]:
text_ = generate_text(model, start_string=u"Я буду долго гнать велосипед ", temperature=0.1)
print(text_)

Я буду долго гнать велосипед ней,
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               


In [51]:
text_ = generate_text(model, start_string=u"Я буду долго гнать велосипед ", temperature=0.2)
print(text_)

Я буду долго гнать велосипед ней,
                         Быть может, в мысли нам приходит
                        Средь модных и шут.

                                                                                                                                                                                                                                                                                                                                                                                                        


In [52]:
text_ = generate_text(model, start_string=u"Я буду долго гнать велосипед ", temperature=0.3)
print(text_)

Я буду долго гнать велосипед ней,
                         Унижусь до смиренной прозы;
                        Тогда роман на старый образец:
                        У тетушки княжны Елены
                        Все тот                                                                                                                                                                                                                                                                                                                     


In [53]:
text_ = generate_text(model, start_string=u"Я буду долго гнать велосипед ", temperature=0.4)
print(text_)

Я буду долго гнать велосипед ней,
                         Бритв и про дев; медвертивом снегов ее прогулки тайком в чепце!
                        Как ты, боже! - стынет кровь,
                         Как только вспомню взгляд холодно споры
                        И уж заранее зевал,
                        И тот же шпиц, и текли в глуши,
                        Не так ли, друг? - Ничуть. Куда! -
                         Он пел те дальные страны,
                        Где долго в лоно тишины
                             


In [54]:
text_ = generate_text(model, start_string=u"Я буду долго гнать велосипед ", temperature=0.5)
print(text_)

Я буду долго гнать велосипед недоступных                                                                                               Две нас нет.

                                                                                                                        XXVIII

                        Мой бедный                                                                                                                                                                                                                          


In [55]:
text_ = generate_text(model, start_string=u"Я буду долго гнать велосипед ", temperature=0.6)
print(text_)

Я буду долго гнать велосипед на поединке друга,
                        Дожив без цели, без трудов
                         На каждой станции трактир.

                                 XXXIX

                        Стал вновь читать готовым                                                                                   XLIIIII

                        Но не теперь. Хоть я сердечно
                        В знаком все глядит - и с тобой
                        Под ней черней полк ливрей,
                        Измены вес


In [56]:
text_ = generate_text(model, start_string=u"Я буду долго гнать велосипед ", temperature=0.7)
print(text_)

Я буду долго гнать велосипед ней,
                        Британской музы небылицы
                        Она в альбомы нежных дев,
                        За гробом ожидает нас.
                        Вела расходы, брила лбы,
                         Ходила в баню по субботу блистал ни чувством,
                         Ни поэтическим огнем,
                        Не                      XXXIII

                        Она поэту подарила
                        В супругом чуть не развелась;
                        Она в


кстати, вполне неплохо :)

In [57]:
text_ = generate_text(model, start_string=u"Я буду долго гнать велосипед ", temperature=0.8)
print(text_)

Я буду долго гнать велосипед на поединке друга,
                        Дожив без целит видеть сон Теперь с каким она вниманьем
                         Читает сладостный роман,
                        В нем ранний ветерок
                         Но вскоре гости понемногу
                        Приемы скоро приняла!
                       Кто их перевести.)

                                                        Меня покойница свекровь. -
                       Ее тревожит сновиденье.
                         Не зная, ка


и это тоже вполне )

In [58]:
text_ = generate_text(model, start_string=u"Я буду долго гнать велосипед ", temperature=0.9)
print(text_)

Я буду долго гнать велосипед неприступною {3}.

                               XVIII

                        Одна бы ль, где выдит ей луна.
                         Охоты узы брака несть,
                        С Онегиным желал сердечно
                         С его лелеяла надежда,
                       И мира новый бесценный,
                                     VI

                                      И дедов верный капитал
                        Иль предузнав издалека
                        В кусты припадшего стр


In [59]:
text_ = generate_text(model, start_string=u"Я буду долго гнать велосипед ", temperature=1.0)
print(text_)

Я буду долго гнать велосипед недоступных шелковый"

                                                             XIX

                                Да бродит по лесам одна.

                                    XXII

                              XXVIII

                        Опрятней модного паркет
                        Уж как она без цели браладбивал, уста мои
                                    Читающий мечты свои
                       Предмет бурь уклопать зари возлюбивой
                        Русская душу ни см


In [60]:
text_ = generate_text(model, start_string=u"Я буду долго гнать велосипед ", temperature=1.1)
print(text_)

Я буду долго гнать велосипед ним, одна,
                         Страстей чужих язык мятежный,
                         И молчалива, как небо, голубые,
                            Ол он, и был счастлив.
                        Скажу благородном расстоянье;
                        Не так небу Преданный конес!
                        Забыв на время все на свете,
                         Она пред хладною толпой,
                        Где нас нет.

                               X

                        Девиты шепчут в угол


In [62]:
text_ = generate_text(model, start_string=u"Я буду долго гнать велосипед ", temperature=1.2)
print(text_)

Я буду долго гнать велосипед нам;
                          Трещит лучинка перед ней.

                               в гостиной встреча новых лиц,
                        Шавнца, бледна,
                        Две сосны корнями срослись;
                        Лая модных на Мрандися,
                         Вела       Все успокоилась кажут бады зсем,
                        На красных лапках гусь тяжелый,
              К реке сквозь меня вечер день морозонсням;
                          Петровский замок. Мрачно он
     


In [63]:
text_ = generate_text(model, start_string=u"Я буду долго гнать велосипед ", temperature=1.3)
print(text_)

Я буду долго гнать велосипед нагой.

                   А Для него измуши мне: нежной,
                        В своей коляскрывнительном,
  на уме,
                        И в необдумаснув пистолет,
                        уле
                         Пуг слушк   Разрешалася с           Заря от уфреплотил!
                        Не умер в правди поет
              XXIII

                        Мартын Задека не решу.

                                        И ей душу по рабно я толку ваш выслушала
                       В


In [64]:
text_ = generate_text(model, start_string=u"Я буду долго гнать велосипед ", temperature=1.4)
print(text_)

Я буду долго гнать велосипед нам на сцянет
                      В душе не презирать людей;
                         Пугая робкую любовь,
                        И субно в салиш сТамки примотритый дом бегом,
 Охлади
                        Укажет будущий невежда
                       На этом глуп, лаждазначен строгою судьбу,
                        Прошу мою
                        У баратился он задумчив Мой Петвились шаже дубав задрепает
     Зыманье и вредн      Сракавый неправилахрадова.
                        Sed l'g


In [65]:
text_ = generate_text(model, start_string=u"Я буду долго гнать велосипед ", temperature=1.5)
print(text_)

Я буду долго гнать велосипед нежных.
                        Онегин сбибший свита родная ведьмоей лодной,
                   В нем думы роем {фки
       Как пьяный путник на него...
                        Уж темно: Танясь нахала;
        -
                        Пря  Не может; дверь толкнул Евгений:
                         Из-за стола, гремя! я след
                       Печальных много приключений.
                    А то, скучая наслажденьем,
                        Храпит - и путник осторожный
                      


Мне больше всего понравилась генерация при температуре 0.7-0.8